In [1]:
import pandas as pd
import numpy as np
import math
import string
import copy
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from collections import Counter
from tqdm import tqdm
from tensorflow_addons.metrics import F1Score
import warnings
import os
warnings.filterwarnings("ignore")

In [41]:
stfold = StratifiedKFold(n_splits=5, shuffle=True)
train_l = list()
val_l = list()
test_l = list()
maxlen = 100
cnt = 0
for train_index, test_index in stfold.split(data['comment'], data['label']):
    cnt += 1
    train = data.iloc[train_index]
    train_x, train_y = train['comment'], train['label']
    tmp = data.iloc[test_index]

    val, test = train_test_split(tmp, test_size=0.5, stratify=tmp['label'])    
    val_x, val_y = val['comment'], val['label']
    test_x, test_y = test['comment'], test['label']    
    
    train_x = keras.preprocessing.sequence.pad_sequences(train_x, maxlen=maxlen)
    val_x = keras.preprocessing.sequence.pad_sequences(val_x, maxlen=maxlen)
    test_x = keras.preprocessing.sequence.pad_sequences(test_x, maxlen=maxlen)
    
    embed_dim = 32  # Embedding size for each token
    vocab_size = n_word_unique + 1

    inputs = layers.Input(shape=(maxlen,))
    embedding_layer = layers.Embedding(input_dim = vocab_size, output_dim = embed_dim, input_length=maxlen)
    x = embedding_layer(inputs)
    x = layers.Bidirectional(layers.LSTM(32, input_dim=(None, embed_dim), 
                                         return_sequences=False, activation='tanh', 
                                         dropout=0.4))(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)

    model = keras.Model(inputs=inputs, outputs=outputs)
    model_path = "lstm_models/{itr:04}/".format(itr=cnt)
    checkpoint_path = model_path + "{epoch:04d}.ckpt"
    checkpoint_dir = os.path.dirname(checkpoint_path)
    !mkdir $model_path
    
    model.compile('adam', "binary_crossentropy", metrics=["accuracy", 
                                                          tf.keras.metrics.AUC(name='auc')])
    
    
    #create checkpoint to save model
    #with best validation loss
    model.save_weights(checkpoint_path.format(epoch=0))

    checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_loss', verbose=1,
            save_weights_only=True, save_best_only=True, mode='auto')
    
    history = model.fit(
                        train_x, train_y, 
                        batch_size=128, epochs=10, 
                        validation_data=(val_x, val_y),
                        callbacks = [checkpoint]
                        )
    
    latest = tf.train.latest_checkpoint(checkpoint_dir)
    model.load_weights(latest)
    
    train_eval = model.evaluate(train_x, train_y)
    val_eval = model.evaluate(val_x, val_y)
    test_eval = model.evaluate(test_x, test_y)
    
    test_l.append(test_eval)
    val_l.append(val_eval)
    train_l.append(train_eval)

mkdir: cannot create directory ‘lstm_models/0001/’: No such file or directory
Epoch 1/10
63/63 [==============================] - ETA: 0s - loss: 0.6808 - accuracy: 0.5500 - auc: 0.5882
Epoch 00001: val_loss improved from inf to 0.66747, saving model to lstm_models/0001/0001.ckpt
63/63 [==============================] - 5s 86ms/step - loss: 0.6808 - accuracy: 0.5500 - auc: 0.5882 - val_loss: 0.6675 - val_accuracy: 0.5800 - val_auc: 0.6319
Epoch 2/10
63/63 [==============================] - ETA: 0s - loss: 0.6062 - accuracy: 0.6875 - auc: 0.7662
Epoch 00002: val_loss improved from 0.66747 to 0.57036, saving model to lstm_models/0001/0002.ckpt
63/63 [==============================] - 5s 73ms/step - loss: 0.6062 - accuracy: 0.6875 - auc: 0.7662 - val_loss: 0.5704 - val_accuracy: 0.7050 - val_auc: 0.7818
Epoch 3/10
63/63 [==============================] - ETA: 0s - loss: 0.3152 - accuracy: 0.8798 - auc: 0.9510
Epoch 00003: val_loss improved from 0.57036 to 0.47486, saving model to lstm_mod

In [42]:
test_l = np.array(test_l)
val_l = np.array(val_l)
train_l = np.array(train_l)

print("test avg loss: ", np.mean(test_l[:, 0]), "+/-" ,np.std(test_l[:, 0]))
print("test avg acc: ", np.mean(test_l[:, 1]),  "+/-" ,np.std(test_l[:, 1]))
print("test avg auc: ", np.mean(test_l[:, 2]),  "+/-" ,np.std(test_l[:, 2]))

print('\n')


print("val avg loss: ", np.mean(val_l[:, 0]), "+/-" ,np.std(val_l[:, 0]))
print("val avg acc: ", np.mean(val_l[:, 1]),  "+/-" ,np.std(val_l[:, 1]))
print("val avg auc: ", np.mean(val_l[:, 2]),  "+/-" ,np.std(val_l[:, 2]))

print('\n')


print("train avg loss: ", np.mean(train_l[:, 0]), "+/-" ,np.std(train_l[:, 0]))
print("train avg acc: ", np.mean(train_l[:, 1]),  "+/-" ,np.std(train_l[:, 1]))
print("train avg auc: ", np.mean(train_l[:, 2]),  "+/-" ,np.std(train_l[:, 2]))


test avg loss:  0.4315872132778168 +/- 0.031824080950908454
test avg acc:  0.8101999878883361 +/- 0.020301727627306546
test avg auc:  0.8838917255401612 +/- 0.017498022538040416


val avg loss:  0.4259478271007538 +/- 0.019996978370534445
val avg acc:  0.8156000018119812 +/- 0.013632309034963812
val avg auc:  0.8856491208076477 +/- 0.012227612609914774


train avg loss:  0.1070766031742096 +/- 0.028616531605469262
train avg acc:  0.9753499865531922 +/- 0.008747643017628726
train avg auc:  0.9961004853248596 +/- 0.0013572799679081008
